In [6]:
using Pkg
Pkg.activate(".")

using JLD2, Plots, Lux, QuadGK, ComponentArrays

  Activating project at `~/code/phasor_julia`


In [19]:
om1 = load("parameters/mlp_id_47_epoch_100.jld2")
om2 = load("parameters/mlp_id_46_epoch_100.jld2")

Dict{String, Any} with 2 entries:
  "params" => (layer_1 = (scale = Float32[0.944844, 1.20831, 1.86019, 1.60119, …
  "state"  => (layer_1 = (running_mean = [0.575794, 0.684621, 3.87745, 16.4577,…

In [13]:
o1 = load("parameters/pmlp_id_47_epoch_100.jld2")
o2 = load("parameters/pmlp_id_46_epoch_100.jld2")

Dict{String, Any} with 2 entries:
  "params" => (layer_1 = (scale = Float32[0.504919, 0.775949, 0.974304, 0.83106…
  "state"  => (layer_1 = (running_mean = [0.575794, 0.684621, 3.87745, 16.4577,…

In [14]:
o1["params"].layer_3

(weight = Float32[0.0728617 -0.040645365 … 0.015499597 -0.21885511; 0.037976142 -0.042849857 … 0.04712785 -0.011631934; … ; 0.14543737 0.03440691 … 0.24273153 -0.06848091; 0.057068672 0.04530454 … 0.010457713 0.055157185], bias = ComplexF32[0.9855259f0 - 0.04457264f0im, 1.0279659f0 + 0.036820382f0im, 0.971679f0 - 0.15884493f0im, 0.964766f0 - 0.048795227f0im, 0.99927264f0 - 0.013948907f0im, 0.982745f0 + 0.031601153f0im, 0.9978303f0 - 0.050703507f0im, 0.93568486f0 - 0.21462259f0im, 1.0021936f0 + 0.01571943f0im, 1.0114331f0 + 0.1293163f0im  …  1.0300401f0 + 0.0036388887f0im, 0.92307717f0 - 0.083539f0im, 0.9856309f0 - 0.026884843f0im, 0.9473845f0 + 0.04764745f0im, 1.0077845f0 + 0.1562096f0im, 0.94417524f0 - 0.12562789f0im, 0.9969729f0 + 0.012071221f0im, 0.9946528f0 + 0.084962375f0im, 1.0248247f0 - 0.029582636f0im, 1.0131812f0 - 0.062435858f0im])

In [29]:
o2["params"].layer_3.weight .- o1["params"].layer_3.weight

128×14 Matrix{Float32}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮              

In [27]:
om1["params"].layer_3.weight .- om2["params"].layer_3.weight

128×14 Matrix{Float32}:
 -0.0355003  -0.311432     0.150665    0.0288978  …   0.340837     1.55281
  0.256593    0.0630805    0.304648   -0.423203      -0.15406      0.524112
  0.0214683  -0.0232079    0.0716589   0.581931      -0.16436      0.754398
  0.205087    0.339076     0.0250235   0.152905       0.144822     0.31259
  0.457026   -0.0563541   -0.0796277   0.0567557      0.395554     0.0155584
  0.0711542   0.301285    -0.536588   -0.548371   …   0.109848     0.545903
  0.177637   -0.0858965   -0.215395   -0.0422139      0.159381    -0.028625
  0.0919949   0.0263786   -0.608622    0.658275       0.0427822    1.00602
 -0.181213    0.351938     0.0772643   0.0652937     -0.1221      -0.582947
  0.444879    0.0360444   -0.0758881   0.183238      -0.00835448  -0.670599
 -0.467171   -0.313563    -0.625247    0.738651   …  -0.325149    -0.217329
 -0.106899   -0.0503475    0.101992   -0.856643       0.109353     0.23798
  0.0092987   0.111544    -0.201491   -0.0224754      0.193155    -